# Feature selection

Training data contains a lot of correlated, redundant, un-trainable or otherwise likely unnecessary features. Plan here is to work through some standard feature selection techniques from scikit-learn to see if we can come up with a good, minimal feature-set to cary forward for other experiments.

In [ ]:
# Change working directory to parent so we can import as we would from __main__.py
print(f'Working directory: ', end = '')
%cd ..

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import mutual_info_classif, SelectKBest, f_classif

import functions.notebook_helper as helper_funcs
import functions.notebook_plotting as plot_funcs
import configuration as config

## 1. Load and prepare data

In [ ]:
# The dataset - omit the file extension, it will be
# added appropriately for the input and output files
dataset_name = 'falcon-7b_scores_v2_10-300_words'

# Input file path
working_hdf5_file = f'{config.DATA_PATH}/{dataset_name}_stage_I_experimental.h5'

# Open a connection to the hdf5 dataset via PyTables with Pandas so we can
# load the data from each bin as a dataframe
data_lake = pd.HDFStore(working_hdf5_file)

# Load bin data
data_df = data_lake[f'master']

# Take small sample for rapid development and testing
data_df = data_df.sample(n = 5000)

# Close hdf5 connection
data_lake.close()

data_df.info()
data_df.head()

OK, let's get the data into shape to train a classifier.

In [ ]:
# Split the data into training and testing
training_data_df = data_df.sample(frac = 0.7, random_state = 42)
testing_data_df = data_df.drop(training_data_df.index)

# Set length threshold
training_data_df = training_data_df[training_data_df['Fragment length (words)'] > 50].copy()
testing_data_df = testing_data_df[testing_data_df['Fragment length (words)'] > 50].copy()

# Remove rows containing NAN
training_data_df.dropna(inplace = True)
testing_data_df.dropna(inplace = True)

# Drop unnecessary and un-trainable
feature_drops = [
    'Fragment ID',
    'Source record num',
    'Dataset',
    'Generator',
    'String',
    'Reader time (seconds)',
    'Writer time (seconds)',
    'Reader peak memory (GB)',
    'Writer peak memory (GB)'
]

training_data_df.drop(feature_drops, axis = 1, inplace = True)
testing_data_df.drop(feature_drops, axis = 1, inplace = True)

# Split the data into features and labels
labels_train = training_data_df['Source']
features_train_df = training_data_df.drop('Source', axis = 1)

labels_test = testing_data_df['Source']
features_test_df = testing_data_df.drop('Source', axis = 1)

# Encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(labels_train)
labels_train = label_encoder.transform(labels_train)
labels_test = label_encoder.transform(labels_test)

print(f'Training data: {len(features_train_df)} examples')
print(f'Test data: {len(features_test_df)} examples')

# Grab the original feature names for future reference
feature_column_names = features_train_df.columns
feature_column_names

Next, standard scale the data. This is not strictly necessary for a tree-based classifier, but it will be helpful when eyeballing the data.

In [4]:
features_train_df, features_test_df = helper_funcs.standard_scale_data(features_train_df, features_test_df, feature_column_names)

Take a look at the features we are working with:

In [ ]:
plot_funcs.plot_feature_distributions(features_train_df).show()

In [ ]:
plot_funcs.plot_cross_correlation_matrix(features_train_df).show()

In [ ]:
plot_funcs.plot_scatter_matrix(features_train_df).show()

Just for fun let's add some synthetic features to the dataset:

In [ ]:
poly_features_train, poly_features_test = helper_funcs.add_poly_features(features_train_df, features_test_df)

In [ ]:
spline_features_train, spline_features_test = helper_funcs.add_spline_features(features_train_df, features_test_df)

## 2. Univariate feature selection

### 2.1. Mutual information
#### 2.1.1 Original feature set

In [ ]:
results = mutual_info_classif(features_train_df, labels_train)

univariate_mi_results = {}

for feature, result in zip(feature_column_names, results):
    univariate_mi_results[feature] = result

univariate_mi_results = dict(sorted(univariate_mi_results.items(), key = lambda item: item[1]))

plt.title('Univariate feature mutual information: original features')

plt.barh(
    np.arange(len(univariate_mi_results.values())),
    univariate_mi_results.values(),
    tick_label = list(univariate_mi_results.keys())
)

plt.xlabel('Mutual information (nats)')

plt.show()

#### 2.1.2. Spline synthetic feature set

In [ ]:
results = mutual_info_classif(spline_features_train, labels_train)

univariate_mi_results = {}

for feature, result in enumerate(results):
    univariate_mi_results[feature] = result

univariate_mi_results = dict(sorted(univariate_mi_results.items(), key = lambda item: item[1]))

plt.title('Univariate feature mutual information: spline features')

plt.scatter(
    univariate_mi_results.values(),
    np.arange(len(univariate_mi_results.values()))
)

plt.xlabel('Mutual information (nats)')
plt.ylabel('Feature')

plt.show()

#### 2.1.3. Polynomial synthetic feature set

In [ ]:
results = mutual_info_classif(poly_features_train, labels_train)

univariate_mi_results = {}

for feature, result in enumerate(results):
    univariate_mi_results[feature] = result

univariate_mi_results = dict(sorted(univariate_mi_results.items(), key=lambda item: item[1]))

plt.title('Univariate feature mutual information: polynomial features')

plt.scatter(
    univariate_mi_results.values(),
    np.arange(len(univariate_mi_results.values()))
)

plt.xlabel('Mutual information (nats)')
plt.ylabel('Feature')

plt.show()

### 2.2. F-Test
#### 2.2.1. Original feature set

In [ ]:
selector = SelectKBest(f_classif, k = 4)
selector.fit(features_train_df, labels_train)

univariate_ftest_results = {}

for feature, result in zip(feature_column_names, selector.pvalues_):
    univariate_ftest_results[feature] = result

univariate_ftest_results = dict(reversed(sorted(univariate_ftest_results.items(), key = lambda item: item[1])))

plt.title('Univariate feature F-Test: original features')

plt.barh(
    np.arange(len(univariate_ftest_results.values())),
    univariate_ftest_results.values(),
   
    tick_label=list(univariate_ftest_results.keys())
)
plt.xscale('log')
plt.xlabel('p-value')

plt.show()

#### 2.2.2. Spline synthetic feature set

In [ ]:
selector = SelectKBest(f_classif, k = 4)
selector.fit(spline_features_train, labels_train)

univariate_ftest_results = {}

for feature, result in enumerate(selector.pvalues_):
    univariate_ftest_results[feature] = result

univariate_ftest_results = dict(reversed(sorted(univariate_ftest_results.items(), key = lambda item: item[1])))

plt.title('Univariate feature F-Test: spline features')

plt.scatter(
    univariate_ftest_results.values(),
    np.arange(len(univariate_ftest_results.values()))
)
plt.xscale('log')
plt.xlabel('p-value')
plt.ylabel('Feature')

plt.show()

#### 2.2.3. Polynomial synthetic feature set

In [ ]:
selector = SelectKBest(f_classif, k = 4)
selector.fit(poly_features_train, labels_train)

univariate_ftest_results = {}

for feature, result in enumerate(selector.pvalues_):
    univariate_ftest_results[feature] = result

univariate_ftest_results = dict(reversed(sorted(univariate_ftest_results.items(), key = lambda item: item[1])))

plt.title('Univariate feature F-Test: polynomial features')

plt.scatter(
    univariate_ftest_results.values(),
    np.arange(len(univariate_ftest_results.values()))
)
plt.xscale('log')
plt.xlabel('p-value')
plt.ylabel('Feature')

plt.show()

OK, cool - both methods seem to agree and many of our features are clearly significant. Question is now, do we need all of them? We already know we have some highly correlated/similar features that we could probably afford to loose.

## 3. Recursive feature elimination

In [16]:
cv_folds = 5

### 3.1. Original feature set

In [ ]:
rfecv, cv_results, plt = helper_funcs.recursive_feature_elimination(
    features_train_df = features_train_df,
    labels_train = labels_train,
    cv_folds = cv_folds,
    n_jobs = -1
)

optimal_feature_count = rfecv.n_features_
plt.show()

### 3.2. Spline synthetic feature set

In [ ]:
spline_rfecv, spline_cv_results, spline_plt = helper_funcs.recursive_feature_elimination(
    features_train_df = spline_features_train, 
    labels_train = labels_train, 
    cv_folds = cv_folds,
    n_jobs = -1
)

optimal_spline_feature_count = spline_rfecv.n_features_
spline_plt.show()

### 3.3. Polynomial synthetic feature set

In [ ]:
poly_rfecv, poly_cv_results, poly_plt = helper_funcs.recursive_feature_elimination(
    features_train_df = poly_features_train, 
    labels_train = labels_train, 
    cv_folds = cv_folds,
    n_jobs = -1
)

optimal_poly_feature_count = poly_rfecv.n_features_
poly_plt.show()

## 4. Reduced feature set evaluation

After a bunch of fiddling around, it seems like the sweet spot for number of features is around 7 or 8 for the original set and 30 or so for the polynomial set. Though, this seems highly dependent on the number of cross-validation folds and the split method. Also, *RFECV* and *SequentialFeatureSelector* seem to often disagree on exactly what those features should be. I think it's time to generate datasets with those numbers of features and train and evaluate some classifiers.

In [20]:
# Make evaluation metrics scorers
scoring = {
    'binary_cross_entropy': make_scorer(helper_funcs.binary_cross_entropy), 
    'accuracy': make_scorer(helper_funcs.percent_accuracy),
    'false_positive_rate': make_scorer(helper_funcs.false_positive_rate),
    'false_negative_rate': make_scorer(helper_funcs.false_negative_rate)
}

# Dictionary to hold results
results = {
    'Fold': [],
    'Condition': [],
    'Fit time (sec.)': [],
    'Accuracy (%)': [],
    'False positive rate': [],
    'False negative rate': [],
    'Binary cross-entropy': []
}

# Plots to draw
plots = ['Fit time (sec.)', 'Accuracy (%)', 'False positive rate', 'False negative rate', 'Binary cross-entropy']

### 4.1. Original feature set

In [ ]:
# The feature set sizes to test
feature_set_sizes = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

# Cross validation folds to run
cv_folds = 5

# Loop on the feature set sizes
for feature_set_size in feature_set_sizes:

    # Run sequential feature selection for this feature set size
    sfs, fitted_sfs = helper_funcs.sequential_feature_selection(
        features_train = features_train_df, 
        labels_train = labels_train, 
        feature_count = feature_set_size, 
        cv_folds = cv_folds,
        n_jobs = -1
    )

    feature_set_train = fitted_sfs.transform(features_train_df)
    print(f'Data shape: {feature_set_train.shape}')

    # Instantiate an XGBoost model using the sklearn API
    model = XGBClassifier()

    # Run cross-validation
    scores = cross_validate(
        model,
        feature_set_train,
        labels_train,
        cv = cv_folds,
        n_jobs = -1,
        scoring = scoring
    )

    # Collect the results
    results = helper_funcs.add_cv_scores(results, scores, f'{feature_set_size} features')

plot_funcs.plot_cross_validation(plots, results).show()

### 4.2. Spline synthetic feature set

In [22]:
# Clear the results dictionary
results = {
    'Fold': [],
    'Condition': [],
    'Fit time (sec.)': [],
    'Accuracy (%)': [],
    'False positive rate': [],
    'False negative rate': [],
    'Binary cross-entropy': []
}

In [ ]:
# The feature set sizes to test
feature_set_sizes = np.arange(optimal_spline_feature_count - 5, optimal_spline_feature_count + 5)

# Loop on the feature set sizes
for feature_set_size in feature_set_sizes:

    # Run sequential feature selection for this feature set size
    sfs, fitted_sfs = helper_funcs.sequential_feature_selection(
        features_train = spline_features_train, 
        labels_train = labels_train, 
        feature_count = feature_set_size, 
        cv_folds = cv_folds,
        n_jobs = -1
    )

    feature_set_train = fitted_sfs.transform(spline_features_train)
    print(f'Data shape: {feature_set_train.shape}')

    # Instantiate an XGBoost model using the sklearn API
    model = XGBClassifier()

    # Run cross-validation
    scores = cross_validate(
        model,
        feature_set_train,
        labels_train,
        cv = cv_folds,
        n_jobs = -1,
        scoring = scoring
    )

    # Collect the results
    results = helper_funcs.add_cv_scores(results, scores, f'{feature_set_size} features')

plot_funcs.plot_cross_validation(plots, results).show()

### 4.3. Polynomial synthetic feature set

In [24]:
# Clear the results dictionary
results = {
    'Fold': [],
    'Condition': [],
    'Fit time (sec.)': [],
    'Accuracy (%)': [],
    'False positive rate': [],
    'False negative rate': [],
    'Binary cross-entropy': []
}

In [25]:
# The feature set sizes to test
feature_set_sizes = np.arange(optimal_poly_feature_count - 5, optimal_poly_feature_count + 5)

# Loop on the feature set sizes
for feature_set_size in feature_set_sizes:

    # Run sequential feature selection for this feature set size
    sfs, fitted_sfs = helper_funcs.sequential_feature_selection(
        features_train = poly_features_train, 
        labels_train = labels_train, 
        feature_count = feature_set_size, 
        cv_folds = cv_folds,
        n_jobs = -1
    )

    feature_set_train = fitted_sfs.transform(poly_features_train)
    print(f'Data shape: {feature_set_train.shape}')

    # Instantiate an XGBoost model using the sklearn API
    model = XGBClassifier()

    # Run cross-validation
    scores = cross_validate(
        model,
        feature_set_train,
        labels_train,
        cv = cv_folds,
        n_jobs = -1,
        scoring = scoring
    )

    # Collect the results
    results = helper_funcs.add_cv_scores(results, scores, f'{feature_set_size} features')

plot_funcs.plot_cross_validation(plots, results).show()